# Testing the analysis 






## First generate the perturbed intensity data
This is done using `compute_series_perp` module to get the true intensity. Then create a wavenumber-dependent sensitivity as a polynomial. Then, perturbed the true intensity and add some noise. This creates a model test data.



In [2]:
import compute_series_perp
import boltzmann_popln as bp
import numpy as np

OJ_H2 = 3
QJ_H2 = 4

OJ_HD = 3
QJ_HD = 3
SJ_HD = 2

OJ_D2 = 4
QJ_D2 = 6
SJ_D2 = 3

# Constants ------------------------------
# these are used for scaling the coefs
scale1 = 1e3
scale2 = 1e6
scale3 = 1e9
scale4 = 1e12
scale5= 1e13
# ----------------------------------------


In [3]:


# perturbation testing

TK = 299
sosD2 = bp.sumofstate_D2(TK)
sosHD = bp.sumofstate_HD(TK)
sosH2 = bp.sumofstate_H2(TK)

computed_D2 = compute_series_perp.spectra_D2(TK, OJ_D2, QJ_D2,
                                                 SJ_D2, sosD2)
computed_HD = compute_series_perp.spectra_HD(TK, OJ_HD, QJ_HD,
                                                 SJ_HD, sosHD)
computed_H2 = compute_series_perp.spectra_H2_c(TK, OJ_H2,
                                                   QJ_H2, sosH2)


# remove row for Q(J=0) --
i, = np.where(computed_D2[:,0] == 0.0)
row_index = np.amin(i)
computed_D2 = np.delete(computed_D2, (row_index), axis=0)
    
i, = np.where(computed_HD[:,0] == 0.0)
row_index = np.amin(i)
computed_HD = np.delete(computed_HD, (row_index), axis=0)
    
i, = np.where(computed_H2[:,0] == 0.0)
row_index = np.amin(i)
computed_H2 = np.delete(computed_H2, (row_index), axis=0)  


##########################################################


def gen_perturbation_factor_linear(c1,xaxis):
    return 1+(c1/scale1)*(xaxis )


def gen_perturbation_factor_quadratic(c1, c2 , xaxis):
    return 1+(c1/scale1)*(xaxis ) + (c2/scale2 )*(xaxis )**2


def gen_perturbation_factor_cubic(c1, c2 , c3, xaxis):
    return 1+(c1/scale1)*(xaxis ) + (c2/scale2 )*(xaxis )**2 + (c3/scale3)*(xaxis )**3

##########################################################

spectral_int_H2 = computed_H2[:, 2]
spectral_int_HD = computed_HD[:, 2]
spectral_int_D2 = computed_D2[:, 2]

scenter = 3316.3
#print(computed_D2)


wavenum_H2 = computed_H2[:,1] - scenter
wavenum_HD = computed_HD[:,1] - scenter
wavenum_D2 = computed_D2[:,1] - scenter



---

### Linear 

In [4]:

print("\n --------- LINEAR ----------- ")

c1 = 0.075


pH2 = gen_perturbation_factor_linear(c1, wavenum_H2 )
pHD = gen_perturbation_factor_linear(c1, wavenum_HD )
pD2 = gen_perturbation_factor_linear(c1, wavenum_D2 )


print(pH2)
print(pHD) 
print (pD2)

print("\n -------------------- ")
noiseH2 = 0.0185*np.random.randn(spectral_int_H2.shape[0])
noiseHD = 0.0175*np.random.randn(spectral_int_HD.shape[0])
noiseD2 = 0.0165*np.random.randn(spectral_int_D2.shape[0])


print("------------------------")

pert_i_H2 = pH2 * spectral_int_H2 
pert_i_HD = pHD * spectral_int_HD
pert_i_D2 = pD2 * spectral_int_D2

####################################################

#print(pert_i_H2)
#print(spectral_int_H2)

#print("------------------------")

#print(pert_i_HD)
#print(spectral_int_HD)

#print("------------------------")

#print(pert_i_D2)
#print(spectral_int_D2)

#print("------------------------\n\n")

perturbed_data_H2 = np.zeros((spectral_int_H2.shape[0],2))
perturbed_data_HD = np.zeros((spectral_int_HD.shape[0],2))
perturbed_data_D2 = np.zeros((spectral_int_D2.shape[0],2))
#-----------------------------------

# noise vectors -------
noise_H2 = 0.0175*np.random.randn(spectral_int_H2.shape[0])
noise_HD = 0.0175*np.random.randn(spectral_int_HD.shape[0])
noise_D2 = 0.0175*np.random.randn(spectral_int_D2.shape[0])
# ---------------------

perturbed_data_H2[:, 0] = pert_i_H2 * 1000
perturbed_data_H2[:, 1] = 0.25 * np.sqrt(pert_i_H2 * 1000)

perturbed_data_HD[:, 0] = pert_i_HD * 1000
perturbed_data_HD[:, 1] = 0.25 * np.sqrt(pert_i_HD *1000)

perturbed_data_D2[:, 0] = pert_i_D2 * 1000
perturbed_data_D2[:, 1] = 0.25 * np.sqrt(pert_i_D2 * 1000)

#-----------------------------------
#  Export intensities perturbed by linear wavelength
#     dependent sensitivity 

np.savetxt('model_H2_perp_linear.exp', perturbed_data_H2)
np.savetxt('model_HD_perp_linear.exp', perturbed_data_HD)
np.savetxt('model_D2_perp_linear.exp', perturbed_data_D2)

print("\n----- Noise vector vs intensity")

print('Noise :\t', noise_H2, '\nIntensity:\t', pert_i_H2 * 1000 , '\nIntensity (pertrbd.):\t', noise_H2 + pert_i_H2 * 1000)

print("\n\t\t Perturbed intensities (linear) exported as: xx_perp_linear.exp")


 --------- LINEAR ----------- 
[1.01889413 1.03678698 1.05897114 1.06071794 1.0620374  1.06292153]
[0.99016922 1.00365934 1.02196098 1.02282326 1.02340033 1.04285345
 1.05519209 1.06702443]
[0.94422587 0.95332549 0.96236875 0.97250941 0.973442   0.97422356
 0.9748517  0.97532451 0.97564052 0.98875454 0.99716673 1.00532219
 1.01318452]

 -------------------- 
------------------------

----- Noise vector vs intensity
Noise :	 [ 0.00310123  0.00438258 -0.01348249  0.03595682  0.00808344  0.01122454] 
Intensity:	 [ 5.66875772  5.02952385  0.20756116  4.49990856  6.09548056 47.15104649] 
Intensity (pertrbd.):	 [ 5.67185895  5.03390643  0.19407867  4.53586538  6.103564   47.16227103]

		 Perturbed intensities (linear) exported as: xx_perp_linear.exp


---

###  Quadratic

In [5]:

print("\n --------- QUADRATIC ----------- ")

c1 = 0.045
c2 = -0.0150


pH2 = gen_perturbation_factor_quadratic(c1, c2, wavenum_H2 )
pHD = gen_perturbation_factor_quadratic(c1, c2, wavenum_HD )
pD2 = gen_perturbation_factor_quadratic(c1, c2, wavenum_D2 )


print(pH2)
print(pHD) 
print (pD2)

print("\n -------------------- ")
noiseH2 = 0.0185*np.random.randn(spectral_int_H2.shape[0])
noiseHD = 0.0175*np.random.randn(spectral_int_HD.shape[0])
noiseD2 = 0.0165*np.random.randn(spectral_int_D2.shape[0])


print("------------------------")

pert_i_H2 = pH2 * spectral_int_H2 
pert_i_HD = pHD * spectral_int_HD
pert_i_D2 = pD2 * spectral_int_D2

####################################################

print(pert_i_H2)
print(spectral_int_H2)

print("------------------------")

print(pert_i_HD)
print(spectral_int_HD)

print("------------------------")

print(pert_i_D2)
print(spectral_int_D2)

print("------------------------\n\n")

perturbed_data_H2 = np.zeros((spectral_int_H2.shape[0],2))
perturbed_data_HD = np.zeros((spectral_int_HD.shape[0],2))
perturbed_data_D2 = np.zeros((spectral_int_D2.shape[0],2))
#-----------------------------------

# noise vectors -------
noise_H2 = 0.0175*np.random.randn(spectral_int_H2.shape[0])
noise_HD = 0.0175*np.random.randn(spectral_int_HD.shape[0])
noise_D2 = 0.0175*np.random.randn(spectral_int_D2.shape[0])
# ---------------------

perturbed_data_H2[:, 0] = pert_i_H2 * 1000
perturbed_data_H2[:, 1] = 0.25 * np.sqrt(pert_i_H2 * 1000)

perturbed_data_HD[:, 0] = pert_i_HD * 1000
perturbed_data_HD[:, 1] = 0.25 * np.sqrt(pert_i_HD *1000)

perturbed_data_D2[:, 0] = pert_i_D2 * 1000
perturbed_data_D2[:, 1] = 0.25 * np.sqrt(pert_i_D2 * 1000)

#-----------------------------------
#  Export intensities perturbed by linear wavelength
#     dependent sensitivity 

np.savetxt('model_H2_perp_quadratic.exp', perturbed_data_H2)
np.savetxt('model_HD_perp_quadratic.exp', perturbed_data_HD)
np.savetxt('model_D2_perp_quadratic.exp', perturbed_data_D2)

print("\n----- Noise vector vs intensity")

print('Noise :\t', noise_H2, '\nIntensity:\t', pert_i_H2 * 1000 , '\nIntensity (pertrbd.):\t', noise_H2 + pert_i_H2 * 1000)

print("\n\t\t Perturbed intensities (quadratic) exported as: xx_perp_quadratic.exp")


 --------- QUADRATIC ----------- 
[1.01038451 1.01846343 1.0261091  1.02659965 1.0269594  1.02719527]
[0.99384382 1.0021599  1.0118905  1.01230489 1.01258    1.02081496
 1.02499214 1.02823526]
[0.95824018 0.96618593 0.97364495 0.98149036 0.98218433 0.98276233
 0.98322452 0.98357102 0.98380196 0.99291549 0.99827863 1.00311778
 1.00744716]

 -------------------- 
------------------------
[0.00562141 0.00494064 0.00020112 0.00435517 0.00589415 0.04556623]
[0.00556364 0.00485107 0.000196   0.00424232 0.00573942 0.04435986]
------------------------
[0.00574939 0.01029478 0.00471228 0.012726   0.02489697 0.02717597
 0.02874121 0.01577817]
[0.005785   0.01027259 0.0046569  0.01257131 0.02458766 0.02662183
 0.02804042 0.0153449 ]
------------------------
[0.00491982 0.00496165 0.01201307 0.00023342 0.00052388 0.00350706
 0.00431006 0.01539963 0.01140622 0.02253901 0.01410617 0.02107331
 0.00538473]
[0.00513422 0.0051353  0.01233825 0.00023782 0.00053338 0.00356858
 0.0043836  0.01565686 0.011

---

### Cubic

In [6]:

print("\n --------- CUBIC ----------- ")

c1 = 0.475
c2 = -0.020
c3 = +0.0120

pH2 = gen_perturbation_factor_cubic(c1, c2, c3, wavenum_H2 )
pHD = gen_perturbation_factor_cubic(c1, c2, c3, wavenum_HD )
pD2 = gen_perturbation_factor_cubic(c1, c2, c3, wavenum_D2 )

#print(pH2)
#print(pHD) 
#print (pD2)

print("\n -------------------- ")
noiseH2 = 0.0185*np.random.randn(spectral_int_H2.shape[0])
noiseHD = 0.0175*np.random.randn(spectral_int_HD.shape[0])
noiseD2 = 0.0165*np.random.randn(spectral_int_D2.shape[0])


print("------------------------")

pert_i_H2 = pH2 * spectral_int_H2 
pert_i_HD = pHD * spectral_int_HD
pert_i_D2 = pD2 * spectral_int_D2

####################################################

#print(pert_i_H2)
#print(spectral_int_H2)

#print("------------------------")

#print(pert_i_HD)
#print(spectral_int_HD)

#print("------------------------")

#print(pert_i_D2)
#print(spectral_int_D2)

#print("------------------------\n\n")

perturbed_data_H2 = np.zeros((spectral_int_H2.shape[0],2))
perturbed_data_HD = np.zeros((spectral_int_HD.shape[0],2))
perturbed_data_D2 = np.zeros((spectral_int_D2.shape[0],2))
#-----------------------------------

# noise vectors -------
noise_H2 = 0.0175*np.random.randn(spectral_int_H2.shape[0])
noise_HD = 0.0175*np.random.randn(spectral_int_HD.shape[0])
noise_D2 = 0.0175*np.random.randn(spectral_int_D2.shape[0])
# ---------------------

perturbed_data_H2[:, 0] = pert_i_H2 * 1000
perturbed_data_H2[:, 1] = 0.25 * np.sqrt(pert_i_H2 * 1000)

perturbed_data_HD[:, 0] = pert_i_HD * 1000
perturbed_data_HD[:, 1] = 0.25 * np.sqrt(pert_i_HD *1000)

perturbed_data_D2[:, 0] = pert_i_D2 * 1000
perturbed_data_D2[:, 1] = 0.25 * np.sqrt(pert_i_D2 * 1000)

#-----------------------------------
#  Export intensities perturbed by linear wavelength
#     dependent sensitivity 

np.savetxt('model_H2_perp_cubic.exp', perturbed_data_H2)
np.savetxt('model_HD_perp_cubic.exp', perturbed_data_HD)
np.savetxt('model_D2_perp_cubic.exp', perturbed_data_D2)

print("\n----- Noise vector vs intensity")

print('Noise :\t', noise_H2, '\nIntensity:\t', pert_i_H2 * 1000 , '\nIntensity (pertrbd.):\t', noise_H2 + pert_i_H2 * 1000)

print("\n\t\t Perturbed intensities (cubic) exported as: xx_perp_cubic.exp")


 --------- CUBIC ----------- 

 -------------------- 
------------------------

----- Noise vector vs intensity
Noise :	 [ 0.00536278  0.01456219 -0.02167563 -0.0226206   0.01451675 -0.00601095] 
Intensity:	 [ 6.22340369  5.9648174   0.26792631  5.84509877  7.95490085 61.7272747 ] 
Intensity (pertrbd.):	 [ 6.22876647  5.97937959  0.24625068  5.82247817  7.96941761 61.72126375]

		 Perturbed intensities (cubic) exported as: xx_perp_cubic.exp
